In [ ]:
## Two-Class EEG Classification using LSTM-based RNN
## Here, we develop a baseline model for using an RNN to perform time-series prediction for one of two motor
## imagery (MI) tasks. The data from the file CLA_SubjectJ-170508-3St-LRHand-Inter.mat is processed and we
## only keep the data points that pertain to the left and right hand MI tasks. Each of the MI task data consists
## of 170 time steps, with each time step consisting of 22 inputs. In other words, we pass the current sample for
## of the 22 channels for each time step. At the end of the 170 time step, we send the output of the LSTM layers
## to a Dense layer with two outputs. To ease the classification at the outer layer, we have converted the class
## into a one-hot encoding of zeros and ones, and use the sigmoid unit at the output to match the output range.

## Preliminary results:
## Performance on training set is 100% classification rate after around 30 to 40 epochs.
## Performance on test set is around 60% - 68%.

## Potential problems:
## The total amount of data is relatively low. Can it be possible to combine multiple subject's L/R Hand MI data
## into one unified data set?
## Can also possible increase the size of the data set by doing some data augmentations.
## For example, injecting some noise into the existing EEG data to create more test data?

## Followup/TODO:
## Use pre-implemented common spatial patterns for feature extraction: avialable here: https://github.com/spolsley/common-spatial-patterns
## Try using a relatively simple non-linear energy operator as a pre-emphasis step on the data to see if it helps generalize
## the model.

In [1]:
## This file is used to split data into series of arrays and their corresponding MI task.
import scipy.io as sio
import matplotlib.pyplot as plt
import numpy as np
file = sio.loadmat('../mat_files/CLA-SubjectJ-170508-3St-LRHand-Inter.mat') #replace with .mat file name
header=file['__header__']
version=file['__version__']
glob=file['__globals__']
ans=file['ans']


x=file['x']
o=file['o'][0][0]
data=o['data']
data = np.transpose(data)
print(data)
nS=o['nS'][0][0]
#values of structure seem to be 2D numpy arrays, if originally a scalar in Matlab.
#use [0][0] to get scalar.
print("Number of samples: {numSamples}".format(numSamples=nS))
test=o['id'][0] #id value became a 1D array of size 1 for some reason. use [0] to get value
print("Dataset ID: {id}".format(id=test))
chnames=o['chnames'][:,0] #[:,0] converts from 2D array back to 1D array
print("Channel names: {channelNames}".format(channelNames=chnames))
markers = o['marker']
## The markers are all still individual arrays of size 1x1, so we convert them to an array with single values
markersArray = []
for marker in markers:
    markersArray.append(marker[0])
markersArray = np.asarray(markersArray)
#For this dataset, the markers are 0, 1, or 2.
# 1 - Left Hand MI, 2 - Right Hand MI, 3 - Passive State, 0 - Rest (???)

[[-18.   -3.6  -6.6 ...  -9.   -7.2  -2.4]
 [-19.2  -0.   -8.4 ...  -8.4 -11.4  -9. ]
 [-12.    1.8  -1.2 ...   2.4   3.6   5.4]
 ...
 [ -6.    5.4   3.  ...   5.4   4.2   3.6]
 [ -8.4   7.2   3.  ...   4.8   6.6   6. ]
 [ -1.2  -1.2  -1.8 ...  -0.   -0.   -0. ]]
Number of samples: 621892
Dataset ID: 201705081338.32BEA9DD
Channel names: [array(['Fp1'], dtype='<U3') array(['Fp2'], dtype='<U3')
 array(['F3'], dtype='<U2') array(['F4'], dtype='<U2')
 array(['C3'], dtype='<U2') array(['C4'], dtype='<U2')
 array(['P3'], dtype='<U2') array(['P4'], dtype='<U2')
 array(['O1'], dtype='<U2') array(['O2'], dtype='<U2')
 array(['A1'], dtype='<U2') array(['A2'], dtype='<U2')
 array(['F7'], dtype='<U2') array(['F8'], dtype='<U2')
 array(['T3'], dtype='<U2') array(['T4'], dtype='<U2')
 array(['T5'], dtype='<U2') array(['T6'], dtype='<U2')
 array(['Fz'], dtype='<U2') array(['Cz'], dtype='<U2')
 array(['Pz'], dtype='<U2')]


In [2]:
## Find the starting indeces where the marker changes
changeIdxs = np.where(np.transpose(markersArray)[:-1] != np.transpose(markersArray)[1:])[0]
print("Number of index changes: {idxChanges}".format(idxChanges=changeIdxs.shape[0]))
## Split the data so that it has its matching marker
dataSplit = np.array_split(data, changeIdxs[:-1], axis=1)
splitCount = 0
for splitData in dataSplit:
    splitCount += 1
print("Number of arrays in data split: {num}".format(num=splitCount))
## Retrieve the marker values for each of the change indeces (changeIdxs)
markerTargets = markersArray[changeIdxs];
print("Number of marker targets: {numTargets}".format(numTargets=markerTargets.shape[0]))

Number of index changes: 1800
Number of arrays in data split: 1800
Number of marker targets: 1800


In [3]:
## To Apply CSP, we first only get the indeces for MI tasks 1 and 2 (left and right hand, respectively.)
LeftIdxs = np.where(markerTargets == 1)
RightIdxs = np.where(markerTargets == 2)
numLeftIdx = LeftIdxs[0].shape

In [4]:
leftData = [];
for leftIndex in LeftIdxs[0]:
    #print(leftIndex)
    #print("Dimensions of index: {ind}".format(ind=dataSplit[leftIndex].shape))
    if(dataSplit[leftIndex].shape[1] != 170):
        continue
    else:
        leftData.append(np.transpose(dataSplit[leftIndex]))
leftData = np.asarray(leftData)
leftData.shape

(288, 170, 22)

In [5]:
rightData = [];
for rightIndex in RightIdxs[0]:
    #print(leftIndex)
    #print("Dimensions of index: {ind}".format(ind=dataSplit[leftIndex].shape))
    if(dataSplit[rightIndex].shape[1] != 170):
        continue
    else:
        rightData.append(np.transpose(dataSplit[rightIndex]))
rightData = np.asarray(rightData)
rightData.shape

(327, 170, 22)

In [6]:
## Only keep the top 288 samples, so that left and right data are equal
rightDataSub = rightData[1:289]

In [7]:
#Construct the target array and merge the data
leftTargets = np.tile(np.array([1,0]),(288,1))
rightTargets = np.tile(np.array([0,1]), (288,1))
markerTargets = np.vstack((leftTargets, rightTargets))
lrData = np.vstack((leftData, rightDataSub))

#Sanity Check
print("lrData Shape: {arg1}\tmarkerTargets Shape: {arg2}".format(arg1=lrData.shape, arg2=markerTargets.shape))

lrData Shape: (576, 170, 22)	markerTargets Shape: (576, 2)


In [8]:
## Construct LSTM using Tensorflow + Keras
# Import Libraries
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import SimpleRNN
from tensorflow.keras.layers import Dropout
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
from tensorflow.keras import optimizers

In [9]:
## Shuffle the data
lrData, markerTargets = shuffle(lrData, markerTargets, random_state=0)

In [10]:
## Split into train and test sets
lrDataTrain, lrDataTest, markerTargetsTrain, markerTargetsTest = train_test_split(lrData, markerTargets, test_size=0.3, random_state=1)
markerTargetsTrain.shape

(403, 2)

In [11]:
## Reshape the data for time-series processing
## Syntax np.reshape((numExamples, numTimeSteps, numInputs/numFeatures))
lrDataTrainRe = lrDataTrain.reshape((lrDataTrain.shape[0], lrDataTrain.shape[1], lrDataTrain.shape[2]))
lrDataTestRe = lrDataTest.reshape((lrDataTest.shape[0], lrDataTest.shape[1], lrDataTest.shape[2]))

In [12]:
## Construct the model
LSTM_EEG = Sequential()
LSTM_EEG.add(LSTM((100),batch_input_shape=(None,lrDataTrainRe.shape[1], lrDataTrainRe.shape[2]), return_sequences=True))
LSTM_EEG.add(LSTM((50), return_sequences=False))
LSTM_EEG.add(Dense((2),activation='sigmoid'))

In [13]:
LSTM_EEG.summary()
sgd = optimizers.SGD(lr=0.05, decay=1e-6, momentum=0.9, nesterov=True)
LSTM_EEG.compile(loss='binary_crossentropy', optimizer=sgd, metrics=['accuracy'])

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 170, 100)          49200     
_________________________________________________________________
lstm_1 (LSTM)                (None, 50)                30200     
_________________________________________________________________
dense (Dense)                (None, 2)                 102       
Total params: 79,502
Trainable params: 79,502
Non-trainable params: 0
_________________________________________________________________


In [14]:
history = LSTM_EEG.fit(lrDataTrain, markerTargetsTrain, epochs=30,verbose=2, batch_size=16)

Epoch 1/30
26/26 - 2s - loss: 0.6788 - accuracy: 0.5806
Epoch 2/30
26/26 - 2s - loss: 0.5982 - accuracy: 0.6700
Epoch 3/30
26/26 - 2s - loss: 0.5424 - accuracy: 0.7122
Epoch 4/30
26/26 - 2s - loss: 0.5325 - accuracy: 0.7370
Epoch 5/30
26/26 - 2s - loss: 0.4419 - accuracy: 0.8040
Epoch 6/30
26/26 - 2s - loss: 0.4109 - accuracy: 0.7965
Epoch 7/30
26/26 - 2s - loss: 0.3802 - accuracy: 0.8337
Epoch 8/30
26/26 - 2s - loss: 0.3880 - accuracy: 0.8213
Epoch 9/30
26/26 - 2s - loss: 0.2569 - accuracy: 0.8908
Epoch 10/30
26/26 - 2s - loss: 0.3459 - accuracy: 0.8486
Epoch 11/30
26/26 - 2s - loss: 0.2786 - accuracy: 0.8710
Epoch 12/30
26/26 - 2s - loss: 0.2127 - accuracy: 0.9156
Epoch 13/30
26/26 - 2s - loss: 0.2770 - accuracy: 0.8883
Epoch 14/30
26/26 - 2s - loss: 0.2545 - accuracy: 0.8983
Epoch 15/30
26/26 - 3s - loss: 0.1220 - accuracy: 0.9653
Epoch 16/30
26/26 - 2s - loss: 0.2056 - accuracy: 0.9280
Epoch 17/30
26/26 - 2s - loss: 0.1693 - accuracy: 0.9256
Epoch 18/30
26/26 - 2s - loss: 0.2190 - 

In [15]:
predictionsTest = LSTM_EEG.predict(lrDataTest)

In [16]:
predictionsTest[predictionsTest>0.5] = 1

In [17]:
predictionsTest[predictionsTest <= 0.5] = 0

In [18]:
comparisonArrayTest = predictionsTest == markerTargetsTest

In [19]:
correctCountTest = 0
for boolValues in comparisonArrayTest:
    if(boolValues[0] & boolValues[1]):
        correctCountTest += 1
falseCountTest = lrDataTest.shape[0] - correctCountTest

predictionsTrain = LSTM_EEG.predict(lrDataTrain)
predictionsTrain[predictionsTrain>0.5] = 1;
predictionsTrain[predictionsTrain<=0.5] = 0;
comparisonArrayTrain = predictionsTrain == markerTargetsTrain;

correctCountTrain = 0
for boolValues in comparisonArrayTrain:
    if(boolValues[0] & boolValues[1]):
        correctCountTrain += 1
falseCountTrain = lrDataTrain.shape[0] - correctCountTrain

In [20]:
print("#################################")
print("#################################")
print("Test Performance:\nCorrect MI Prediction: {}\nIncorrect MI Prediction: {}\nPercent Accuracy: {:.3f}%".format(correctCountTrain, falseCountTrain, (correctCountTrain*100/lrDataTrain.shape[0])))
print("#################################")
print("#################################")
print("Test Performance:\nCorrect MI Prediction: {}\nIncorrect MI Prediction: {}\nPercent Accuracy: {:.3f}%".format(correctCountTest, falseCountTest, (correctCountTest*100/lrDataTest.shape[0])))
print("#################################")
print("#################################")

#################################
#################################
Test Performance:
Correct MI Prediction: 403
Incorrect MI Prediction: 0
Percent Accuracy: 100.000%
#################################
#################################
Test Performance:
Correct MI Prediction: 110
Incorrect MI Prediction: 63
Percent Accuracy: 63.584%
#################################
#################################
